In [1]:
using ITensors 
using Plots
using Measurements   
using DelimitedFiles 

In [2]:
L = 10;
N = 1;
l = L-2;
m = 0.4;
G² = 1.0;
sw = 3;

In [3]:
sites = siteinds("Fermion",L)
ampoₗ = OpSum()
ampoₛ = OpSum()

OpSum:


In [4]:
for j in 1:L-N
    ampoₗ -= 1im,"C",j,"Cdag",j+N
    ampoₗ += 1im,"C",j+N,"Cdag",j
end
for j in 1:N:L-N
    ampoₗ += (G²/2),"C * Cdag",j
end
for j in 1:L
    ampoₗ += (m*(-1)^j),"C * Cdag",j
end
Hₗ = MPO(ampoₗ, sites)
ψₗ₀ = randomMPS(sites,100)
sweep = Sweeps(sw)
setmaxdim!(sweep,repeat([1024],sw)...)
setcutoff!(sweep,1E-8)
eₗ₀,ψₗ = dmrg(Hₗ,ψₗ₀,sweep,outputlevel=0)

(-4.121132429224502, MPS
[1] ((dim=2|id=549|"Link,l=1"), (dim=2|id=62|"Fermion,Site,n=1"))
[2] ((dim=4|id=893|"Link,l=2"), (dim=2|id=25|"Fermion,Site,n=2"), (dim=2|id=549|"Link,l=1"))
[3] ((dim=2|id=905|"Fermion,Site,n=3"), (dim=8|id=869|"Link,l=3"), (dim=4|id=893|"Link,l=2"))
[4] ((dim=2|id=949|"Fermion,Site,n=4"), (dim=14|id=526|"Link,l=4"), (dim=8|id=869|"Link,l=3"))
[5] ((dim=2|id=807|"Fermion,Site,n=5"), (dim=14|id=281|"Link,l=5"), (dim=14|id=526|"Link,l=4"))
[6] ((dim=2|id=242|"Fermion,Site,n=6"), (dim=13|id=997|"Link,l=6"), (dim=14|id=281|"Link,l=5"))
[7] ((dim=2|id=900|"Fermion,Site,n=7"), (dim=8|id=202|"Link,l=7"), (dim=13|id=997|"Link,l=6"))
[8] ((dim=2|id=192|"Fermion,Site,n=8"), (dim=4|id=184|"Link,l=8"), (dim=8|id=202|"Link,l=7"))
[9] ((dim=2|id=846|"Fermion,Site,n=9"), (dim=2|id=182|"Link,l=9"), (dim=4|id=184|"Link,l=8"))
[10] ((dim=2|id=643|"Fermion,Site,n=10"), (dim=2|id=182|"Link,l=9"))
)

In [5]:
for j in 1:l-N
    ampoₛ -= 1im,"C",j,"Cdag",j+N
    ampoₛ += 1im,"C",j+N,"Cdag",j
end
for j in 1:N:l-N
    ampoₛ += (G²/2),"C * Cdag",j
end
for j in 1:l
    ampoₛ += (m*(-1)^j),"C * Cdag",j 
end
Hₛ = MPO(ampoₛ,sites[1:l])
ψₛ₀ = randomMPS(sites[1:l],100)
sweep = Sweeps(sw)
setmaxdim!(sweep,repeat([1024],sw)...)
setcutoff!(sweep,1E-8)
eₛ₀,ψₛ = dmrg(Hₛ,ψₛ₀,sweep,outputlevel=0)

(-3.263183252416383, MPS
[1] ((dim=2|id=446|"Link,l=1"), (dim=2|id=62|"Fermion,Site,n=1"))
[2] ((dim=4|id=264|"Link,l=2"), (dim=2|id=25|"Fermion,Site,n=2"), (dim=2|id=446|"Link,l=1"))
[3] ((dim=2|id=905|"Fermion,Site,n=3"), (dim=8|id=886|"Link,l=3"), (dim=4|id=264|"Link,l=2"))
[4] ((dim=2|id=949|"Fermion,Site,n=4"), (dim=13|id=921|"Link,l=4"), (dim=8|id=886|"Link,l=3"))
[5] ((dim=2|id=807|"Fermion,Site,n=5"), (dim=8|id=278|"Link,l=5"), (dim=13|id=921|"Link,l=4"))
[6] ((dim=2|id=242|"Fermion,Site,n=6"), (dim=4|id=681|"Link,l=6"), (dim=8|id=278|"Link,l=5"))
[7] ((dim=2|id=900|"Fermion,Site,n=7"), (dim=2|id=513|"Link,l=7"), (dim=4|id=681|"Link,l=6"))
[8] ((dim=2|id=192|"Fermion,Site,n=8"), (dim=2|id=513|"Link,l=7"))
)

In [28]:
a,b = nothing, nothing

(nothing, nothing)

In [29]:
v1 = ITensor(sites[l+1])
for j in 1:size(v1)[1]
    v1[j]=1/sqrt(2)
end
@show v1;

v1 = ITensor ord=1
Dim 1: (dim=2|id=846|"Fermion,Site,n=9")
NDTensors.Dense{Float64, Vector{Float64}}
 2-element
 0.7071067811865475
 0.7071067811865475


In [30]:
v2 = ITensor(sites[L])
for j in 1:size(v2)[1]
    v2[j] = 1/sqrt(2)
end
@show v2;

v2 = ITensor ord=1
Dim 1: (dim=2|id=643|"Fermion,Site,n=10")
NDTensors.Dense{Float64, Vector{Float64}}
 2-element
 0.7071067811865475
 0.7071067811865475


In [31]:
v = ITensor(1.0)
for j in 1:l
    v = v*dag(ψₛ[j])*ψₗ[j]
end
v = v*v1*ψₗ[l+1]
v = v*v2*ψₗ[L]
@show v;

v = ITensor ord=0
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 0-dimensional
-0.19096570157824214 - 0.4265079896854094im


In [35]:
inner(v,v)

0.21837696444475954 + 0.0im

In [34]:
real(inner(v,v))

0.21837696444475954

In [33]:
norm(v)

0.4673082114030948

In [ ]:

# lattice_large - lattice_small = 2 
function GrossNeveu(L,N,m,G²,sw)
    l = L-2;
    sites = siteinds("Fermion",L)
    ampoₗ = OpSum()
    ampoₛ = OpSum()
    # Large Lattice 
    for j in 1:L-N
        ampoₗ -= 1im,"C",j,"Cdag",j+N
        ampoₗ += 1im,"C",j+N,"Cdag",j
    end
    for j in 1:N:L-N
        ampoₗ += (G²/2),"C * Cdag",j
    end
    for j in 1:L
        ampoₗ += (m*(-1)^j),"C * Cdag",j
    end
    Hₗ = MPO(ampoₗ, sites)
    ψₗ₀ = randomMPS(sites,100)
    ######################################
    ## small lattice 
    for j in 1:l-N
        ampoₛ -= 1im,"C",j,"Cdag",j+N
        ampoₛ += 1im,"C",j+N,"Cdag",j
    end
    for j in 1:N:l-N
        ampoₛ += (G²/2),"C * Cdag",j
    end
    for j in 1:l
        ampoₛ += (m*(-1)^j),"C * Cdag",j 
    end
    Hₛ = MPO(ampoₛ,sites[1:l])
    ψₛ₀ = randomMPS(sites[1:l],100)
    sweep = Sweeps(sw)
    setmaxdim!(sweep,repeat([1024],sw)...)
    setcutoff!(sweep,1E-8)
    
    eₗ₀,ψₗ = dmrg(Hₗ,ψₗ₀,sweep,outputlevel=0)
    eₛ₀,ψₛ = dmrg(Hₛ,ψₛ₀,sweep,outputlevel=0)
    
    #  Overlap computation
    v1 = ITensor(sites[l+1]) 
    for j in 1:size(v1)[1]
        v1[j] = 1/sqrt(2)
    end
    v2 = ITensor(sites[L])
    for j in 1:size(v2)[1]
        v2[j] = 1/sqrt(2)
    end
    v = ITensor(1.0)
    for j in 1:l
        v = v*dag(ψₛ[j])*ψₗ[j]
    end
    v = v*v1*ψₗ[l+1]
    v = v*v2*ψₗ[L]
    return inner(v,v), norm(v)
end

In [ ]:
using ITensors 
using Plots
using Measurements   
using DelimitedFiles 

# lattice_large - lattice_small = 2 
function GrossNeveu(L,N,m,G²,sw)
    l = L-2;
    sites = siteinds("Fermion",L)
    ampoₗ = OpSum()
    ampoₛ = OpSum()
    # Large Lattice 
    for j in 1:L-N
        ampoₗ -= 1im,"C",j,"Cdag",j+N
        ampoₗ += 1im,"C",j+N,"Cdag",j
    end
    for j in 1:N:L-N
        ampoₗ += (G²/2),"C * Cdag",j
    end
    for j in 1:L
        ampoₗ += (m*(-1)^j),"C * Cdag",j
    end
    Hₗ = MPO(ampoₗ, sites)
    ψₗ₀ = randomMPS(sites,100)
    ######################################
    ## small lattice 
    for j in 1:l-N
        ampoₛ -= 1im,"C",j,"Cdag",j+N
        ampoₛ += 1im,"C",j+N,"Cdag",j
    end
    for j in 1:N:l-N
        ampoₛ += (G²/2),"C * Cdag",j
    end
    for j in 1:l
        ampoₛ += (m*(-1)^j),"C * Cdag",j 
    end
    Hₛ = MPO(ampoₛ,sites[1:l])
    ψₛ₀ = randomMPS(sites[1:l],100)
    sweep = Sweeps(sw)
    setmaxdim!(sweep,repeat([1024],sw)...)
    setcutoff!(sweep,1E-8)
    
    eₗ₀,ψₗ = dmrg(Hₗ,ψₗ₀,sweep,outputlevel=0)
    eₛ₀,ψₛ = dmrg(Hₛ,ψₛ₀,sweep,outputlevel=0)
    
    #  Overlap computation
    v1 = ITensor(sites[l+1]) 
    for j in 1:size(v1)[1]
        v1[j] = 1/sqrt(2)
    end
    v2 = ITensor(sites[L])
    for j in 1:size(v2)[1]
        v2[j] = 1/sqrt(2)
    end
    v = ITensor(1.0)
    for j in 1:l
        v = v*dag(ψₛ[j])*ψₗ[j]
    end
    v = v*v1*ψₗ[l+1]
    v = v*v2*ψₗ[L]
    return inner(v,v), norm(v)
end

# L::Integer, N::Integer,G²::Float64,m::Float64,sw::Integer
result_res2 = Vector{ComplexF64}()
result_norm2 = Vector{ComplexF64}()

for j in 4:100
    println(j)
    res2 , norm_res2 = GrossNeveu(j,1,0.7,1.0,3); # change BD --> 100
    push!(result_res2, res2);
    push!(result_norm2, norm_res2);
end   

site_number2 = [x for x in 1:length(result_res2)]
result_res22 = real(result_res2)
result_norm22 = real(result_norm2)
# scatter(site_number2, 
#     result_res22 .± 1E-5, 
#     xlim=(-1,25),xticks=((-1:25)), 
#     ylim=(0.18,0.27), 
#     yticks=((0.18:0.001:0.23)),   
#     markersize=2,
#     xlabel="Site Number",
#     ylabel="Inner Product",
#     title="Overlap")
# savefig("GN_m04.svg")
writedlm("res_m07_100.txt",result_res22)
# writedlm("norm_m07_50.txt",result_norm22)